### Link: https://www.mysqltutorial.org/mysql-window-functions/

In [1]:
import pandas as pd

In [2]:
import mysql.connector
 
# Connecting from the server
conn = mysql.connector.connect(user = 'root',
                               host = 'localhost',
                              database = 'mysql_db',
                              password = 'pass@learning')

In [3]:
cursor = conn.cursor()

In [3]:
'''
cursor.execute("""CREATE TABLE sales(
    sales_employee VARCHAR(50) NOT NULL,
    fiscal_year INT NOT NULL,
    sale DECIMAL(14,2) NOT NULL,
    PRIMARY KEY(sales_employee,fiscal_year)
)""")

cursor.execute("""INSERT INTO sales(sales_employee,fiscal_year,sale)
VALUES('Bob',2016,100),
      ('Bob',2017,150),
      ('Bob',2018,200),
      ('Alice',2016,150),
      ('Alice',2017,100),
      ('Alice',2018,200),
       ('John',2016,200),
      ('John',2017,150),
      ('John',2018,250)""")

conn.commit()
'''

In [4]:
pd.read_sql_query("""select * from sales
                  """ \
                  , conn)

,sales_employee,fiscal_year,sale
0,Alice,2016,150.0
1,Alice,2017,100.0
2,Alice,2018,200.0
3,Bob,2016,100.0
4,Bob,2017,150.0
5,Bob,2018,200.0
6,John,2016,200.0
7,John,2017,150.0
8,John,2018,250.0


In [5]:
pd.read_sql_query("""SELECT SUM(sale) FROM sales
                  """ \
                  , conn)

,SUM(sale)
0,1500.0


In [6]:
pd.read_sql_query("""SELECT fiscal_year, SUM(sale) FROM sales
                    GROUP BY 
                    fiscal_year
                  """ \
                  , conn)

,fiscal_year,SUM(sale)
0,2016,450.0
1,2017,400.0
2,2018,650.0


In [13]:
pd.read_sql_query("""SELECT 
    fiscal_year, 
    sales_employee,
    sale,
    SUM(sale) OVER (PARTITION BY fiscal_year) total_sales
FROM
    sales
                  """ \
                  , conn)

,fiscal_year,sales_employee,sale,total_sales
0,2016,Alice,150.0,450.0
1,2016,Bob,100.0,450.0
2,2016,John,200.0,450.0
3,2017,Alice,100.0,400.0
4,2017,Bob,150.0,400.0
5,2017,John,150.0,400.0
6,2018,Alice,200.0,650.0
7,2018,Bob,200.0,650.0
8,2018,John,250.0,650.0


#### MySQL Window Function list
#### The following shows the window functions in MySQL:

1. CUME_DIST: Calculates the cumulative distribution of a value in a set of values.
2. DENSE_RANK: Assigns a rank to every row within its partition based on the ORDER BY clause. It assigns the same rank to the rows with equal values. If two or more rows have the same rank, then there will be no gaps in the sequence of ranked values.
3. FIRST_VALUE: Returns the value of the specified expression with respect to the first row in the window frame.
4. LAG: Returns the value of the Nth row before the current row in a partition. It returns NULL if no preceding row exists.
5. LAST_VALUE: Returns the value of the specified expression with respect to the last row in the window frame.
6. LEAD: Returns the value of the Nth row after the current row in a partition. It returns NULL if no subsequent row exists.
7. NTH_VALUE: Returns value of argument from Nth row of the window frame
8. NTILE: Distributes the rows for each window partition into a specified number of ranked groups.
9. PERCENT_RANK: Calculates the percentile rank of a row in a partition or result set
10. RANK: Similar to the DENSE_RANK() function except that there are gaps in the sequence of ranked values when two or more rows have the same rank.
11. ROW_NUMBER: Assigns a sequential integer to every row within its partition

In [17]:
cursor.execute("""DROP TABLE if exists scores""")

cursor.execute("""CREATE TABLE scores (
    name VARCHAR(20) PRIMARY KEY,
    score INT NOT NULL
)""")

cursor.execute("""INSERT INTO
scores (name, score)
VALUES
('Smith',81),
('Jones',55),
('Williams',55),
('Taylor',62),
('Brown',62),
('Davies',84),
('Evans',87),
('Wilson',72),
('Thomas',72),
('Johnson',100)""")

conn.commit()

In [18]:
pd.read_sql_query("""SELECT * FROM scores""" \
                  , conn)

,name,score
0,Brown,62
1,Davies,84
2,Evans,87
3,Johnson,100
4,Jones,55
5,Smith,81
6,Taylor,62
7,Thomas,72
8,Williams,55
9,Wilson,72


In [20]:
# CUME_DIST
pd.read_sql_query("""SELECT
    name,
    score,
    ROW_NUMBER() OVER (ORDER BY score) row_num,
    CUME_DIST() OVER (ORDER BY score) cume_dist_val
FROM scores""" \
                  , conn)

,name,score,row_num,cume_dist_val
0,Jones,55,1,0.2
1,Williams,55,2,0.2
2,Brown,62,3,0.4
3,Taylor,62,4,0.4
4,Thomas,72,5,0.6
5,Wilson,72,6,0.6
6,Smith,81,7,0.7
7,Davies,84,8,0.8
8,Evans,87,9,0.9
9,Johnson,100,10,1.0


In [5]:
cursor.execute("""DROP TABLE if exists t""")
cursor.execute("""CREATE TABLE t (val int)""")

cursor.execute("""INSERT INTO t (val)
VALUES (1), (2), (2), (3), (4), (4), (5)""")

conn.commit()


In [7]:
# DENSE_RANK
pd.read_sql_query("""SELECT
    val,
    DENSE_RANK() OVER (
        ORDER BY val
    ) my_dense_rank
FROM
    t""" , conn)



,val,my_dense_rank
0,1,1
1,2,2
2,2,2
3,3,3
4,4,4
5,4,4
6,5,5


In [8]:
# DENSE_RANK
pd.read_sql_query("""SELECT
    sales_employee,
    fiscal_year,
    sale,
    DENSE_RANK() OVER (PARTITION BY
                     fiscal_year
                 ORDER BY
                     sale DESC
                ) sales_rank
FROM
    sales""" , conn)



,sales_employee,fiscal_year,sale,sales_rank
0,John,2016,200.0,1
1,Alice,2016,150.0,2
2,Bob,2016,100.0,3
3,Bob,2017,150.0,1
4,John,2017,150.0,1
5,Alice,2017,100.0,2
6,John,2018,250.0,1
7,Alice,2018,200.0,2
8,Bob,2018,200.0,2


In [9]:
cursor.execute("""DROP TABLE if exists overtime""")
cursor.execute("""CREATE TABLE overtime (
    employee_name VARCHAR(50) NOT NULL,
    department VARCHAR(50) NOT NULL,
    hours INT NOT NULL,
    PRIMARY KEY (employee_name , department)
)""")

cursor.execute("""INSERT INTO overtime(employee_name, department, hours)
VALUES('Diane Murphy','Accounting',37),
('Mary Patterson','Accounting',74),
('Jeff Firrelli','Accounting',40),
('William Patterson','Finance',58),
('Gerard Bondur','Finance',47),
('Anthony Bow','Finance',66),
('Leslie Jennings','IT',90),
('Leslie Thompson','IT',88),
('Julie Firrelli','Sales',81),
('Steve Patterson','Sales',29),
('Foon Yue Tseng','Sales',65),
('George Vanauf','Marketing',89),
('Loui Bondur','Marketing',49),
('Gerard Hernandez','Marketing',66),
('Pamela Castillo','SCM',96),
('Larry Bott','SCM',100),
('Barry Jones','SCM',65)""")

conn.commit()

In [11]:
# FIRST_VALUE
pd.read_sql_query("""SELECT
    employee_name,
    hours,
    FIRST_VALUE(employee_name) OVER (
        ORDER BY hours
    ) least_over_time
FROM
    overtime""" , conn)

,employee_name,hours,least_over_time
0,Steve Patterson,29,Steve Patterson
1,Diane Murphy,37,Steve Patterson
2,Jeff Firrelli,40,Steve Patterson
3,Gerard Bondur,47,Steve Patterson
4,Loui Bondur,49,Steve Patterson
5,William Patterson,58,Steve Patterson
6,Barry Jones,65,Steve Patterson
7,Foon Yue Tseng,65,Steve Patterson
8,Anthony Bow,66,Steve Patterson
9,Gerard Hernandez,66,Steve Patterson


In [13]:
# FIRST_VALUE
pd.read_sql_query("""SELECT
    employee_name,
    department,
    hours,
    FIRST_VALUE(employee_name) OVER (
        PARTITION BY department
        ORDER BY hours
    ) least_over_time
FROM
    overtime""" , conn)

,employee_name,department,hours,least_over_time
0,Diane Murphy,Accounting,37,Diane Murphy
1,Jeff Firrelli,Accounting,40,Diane Murphy
2,Mary Patterson,Accounting,74,Diane Murphy
3,Gerard Bondur,Finance,47,Gerard Bondur
4,William Patterson,Finance,58,Gerard Bondur
5,Anthony Bow,Finance,66,Gerard Bondur
6,Leslie Thompson,IT,88,Leslie Thompson
7,Leslie Jennings,IT,90,Leslie Thompson
8,Loui Bondur,Marketing,49,Loui Bondur
9,Gerard Hernandez,Marketing,66,Loui Bondur


In [5]:
pd.read_sql_query("""SELECT productline,
           YEAR(orderDate) order_year,
           quantityOrdered, priceEach
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)""" , conn)

,productline,order_year,quantityOrdered,priceEach
0,Vintage Cars,2003,30,136.00
1,Vintage Cars,2003,50,55.09
2,Vintage Cars,2003,22,75.46
3,Vintage Cars,2003,49,35.29
4,Vintage Cars,2003,25,108.06
...,...,...,...,...
2991,Trucks and Buses,2005,49,127.79
2992,Classic Cars,2005,31,31.82
2993,Trucks and Buses,2005,41,83.79
2994,Trucks and Buses,2005,11,50.32


In [7]:
# LAG
pd.read_sql_query("""WITH productline_sales AS (
    SELECT productline,
           YEAR(orderDate) order_year,
           ROUND(SUM(quantityOrdered * priceEach),0) order_value
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)
    GROUP BY productline, order_year
)
SELECT
    productline, 
    order_year, 
    order_value,
    LAG(order_value, 1) OVER (
        PARTITION BY productLine
        ORDER BY order_year
    ) prev_year_order_value
FROM 
    productline_sales""" , conn)

,productline,order_year,order_value,prev_year_order_value
0,Classic Cars,2003,1374832.0,NaN
1,Classic Cars,2004,1763137.0,1374832.0
2,Classic Cars,2005,715954.0,1763137.0
3,Motorcycles,2003,348909.0,NaN
4,Motorcycles,2004,527244.0,348909.0
5,Motorcycles,2005,245273.0,527244.0
6,Planes,2003,309784.0,NaN
7,Planes,2004,471971.0,309784.0
8,Planes,2005,172882.0,471971.0
9,Ships,2003,222182.0,NaN


In [34]:
# LEAD
pd.read_sql_query("""SELECT 
    customerName,
    orderDate,
    LEAD(orderDate,1) OVER (
        PARTITION BY customerNumber
        ORDER BY orderDate ) nextOrderDate
FROM 
    orders
INNER JOIN customers USING (customerNumber)""" , conn)

,customerName,orderDate,nextOrderDate
0,Atelier graphique,2003-05-20,2004-09-27
1,Atelier graphique,2004-09-27,2004-11-25
2,Atelier graphique,2004-11-25,None
3,Signal Gift Stores,2003-05-21,2004-08-06
4,Signal Gift Stores,2004-08-06,2004-11-29
...,...,...,...
321,Diecast Collectables,2004-04-26,None
322,Kelly's Gift Shop,2003-07-07,2003-11-11
323,Kelly's Gift Shop,2003-11-11,2004-12-16
324,Kelly's Gift Shop,2004-12-16,2005-04-01


In [28]:
# LAST_VALUE
pd.read_sql_query("""SELECT
    employee_name,
    department,
    hours,
    LAST_VALUE(employee_name) OVER (
    PARTITION BY department
        ORDER BY hours
        RANGE BETWEEN
        UNBOUNDED PRECEDING AND
            UNBOUNDED FOLLOWING
    ) most_overtime_employee
FROM
    overtime""" , conn)

,employee_name,department,hours,most_overtime_employee
0,Diane Murphy,Accounting,37,Mary Patterson
1,Jeff Firrelli,Accounting,40,Mary Patterson
2,Mary Patterson,Accounting,74,Mary Patterson
3,Gerard Bondur,Finance,47,Anthony Bow
4,William Patterson,Finance,58,Anthony Bow
5,Anthony Bow,Finance,66,Anthony Bow
6,Leslie Thompson,IT,88,Leslie Jennings
7,Leslie Jennings,IT,90,Leslie Jennings
8,Loui Bondur,Marketing,49,George Vanauf
9,Gerard Hernandez,Marketing,66,George Vanauf


In [31]:
# LAST_VALUE
pd.read_sql_query("""SELECT
    employee_name,
    department,
    hours,
    LAST_VALUE(employee_name) OVER (
    PARTITION BY department
        ORDER BY hours
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) most_overtime_employee
FROM
    overtime""" , conn)

,employee_name,department,hours,most_overtime_employee
0,Diane Murphy,Accounting,37,Diane Murphy
1,Jeff Firrelli,Accounting,40,Jeff Firrelli
2,Mary Patterson,Accounting,74,Mary Patterson
3,Gerard Bondur,Finance,47,Gerard Bondur
4,William Patterson,Finance,58,William Patterson
5,Anthony Bow,Finance,66,Anthony Bow
6,Leslie Thompson,IT,88,Leslie Thompson
7,Leslie Jennings,IT,90,Leslie Jennings
8,Loui Bondur,Marketing,49,Loui Bondur
9,Gerard Hernandez,Marketing,66,Gerard Hernandez


In [32]:
# LAST_VALUE
pd.read_sql_query("""SELECT
    employee_name,
    department,
    hours,
    LAST_VALUE(employee_name) OVER (
    PARTITION BY department
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) most_overtime_employee
FROM
    overtime""" , conn)

,employee_name,department,hours,most_overtime_employee
0,Diane Murphy,Accounting,37,Mary Patterson
1,Jeff Firrelli,Accounting,40,Mary Patterson
2,Mary Patterson,Accounting,74,Mary Patterson
3,Anthony Bow,Finance,66,William Patterson
4,Gerard Bondur,Finance,47,William Patterson
5,William Patterson,Finance,58,William Patterson
6,Leslie Jennings,IT,90,Leslie Thompson
7,Leslie Thompson,IT,88,Leslie Thompson
8,George Vanauf,Marketing,89,Loui Bondur
9,Gerard Hernandez,Marketing,66,Loui Bondur


In [38]:
cursor.execute("""DROP TABLE if exists basic_pays""")
cursor.execute("""CREATE TABLE basic_pays(
    employee_name VARCHAR(50) NOT NULL,
    department VARCHAR(50) NOT NULL,
    salary INT NOT NULL,
    PRIMARY KEY (employee_name , department)
)""")

cursor.execute("""INSERT INTO 
	basic_pays(employee_name, 
			   department, 
			   salary)
VALUES
	('Diane Murphy','Accounting',8435),
	('Mary Patterson','Accounting',9998),
	('Jeff Firrelli','Accounting',8992),
	('William Patterson','Accounting',8870),
	('Gerard Bondur','Accounting',11472),
	('Anthony Bow','Accounting',6627),
	('Leslie Jennings','IT',8113),
	('Leslie Thompson','IT',5186),
	('Julie Firrelli','Sales',9181),
	('Steve Patterson','Sales',9441),
	('Foon Yue Tseng','Sales',6660),
	('George Vanauf','Sales',10563),
	('Loui Bondur','SCM',10449),
	('Gerard Hernandez','SCM',6949),
	('Pamela Castillo','SCM',11303),
	('Larry Bott','SCM',11798),
	('Barry Jones','SCM',10586)""")
conn.commit()

In [39]:
# NTH_VALUE
pd.read_sql_query("""SELECT
	employee_name,
	department,
	salary,
	NTH_VALUE(employee_name, 2) OVER  (
		PARTITION BY department
		ORDER BY salary DESC
		RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
	) second_highest_salary
FROM
	basic_pays""" , conn)

,employee_name,department,salary,second_highest_salary
0,Gerard Bondur,Accounting,11472,Mary Patterson
1,Mary Patterson,Accounting,9998,Mary Patterson
2,Jeff Firrelli,Accounting,8992,Mary Patterson
3,William Patterson,Accounting,8870,Mary Patterson
4,Diane Murphy,Accounting,8435,Mary Patterson
5,Anthony Bow,Accounting,6627,Mary Patterson
6,Leslie Jennings,IT,8113,Leslie Thompson
7,Leslie Thompson,IT,5186,Leslie Thompson
8,George Vanauf,Sales,10563,Steve Patterson
9,Steve Patterson,Sales,9441,Steve Patterson


In [40]:
# NTH_VALUE
pd.read_sql_query("""SELECT
	employee_name,
	department,
	salary,
	NTH_VALUE(employee_name, 2) OVER  (
		PARTITION BY department
		ORDER BY salary DESC
		RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
	) second_highest_salary
FROM
	basic_pays""" , conn)

,employee_name,department,salary,second_highest_salary
0,Gerard Bondur,Accounting,11472,None
1,Mary Patterson,Accounting,9998,Mary Patterson
2,Jeff Firrelli,Accounting,8992,Mary Patterson
3,William Patterson,Accounting,8870,Mary Patterson
4,Diane Murphy,Accounting,8435,Mary Patterson
5,Anthony Bow,Accounting,6627,Mary Patterson
6,Leslie Jennings,IT,8113,None
7,Leslie Thompson,IT,5186,Leslie Thompson
8,George Vanauf,Sales,10563,None
9,Steve Patterson,Sales,9441,Steve Patterson


In [42]:
# NTH_VALUE
pd.read_sql_query("""SELECT
	employee_name,
	department,
	salary,
	NTH_VALUE(employee_name, 2) OVER  (
		PARTITION BY department
		RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
	) second_highest_salary
FROM
	basic_pays""" , conn)

,employee_name,department,salary,second_highest_salary
0,Anthony Bow,Accounting,6627,Diane Murphy
1,Diane Murphy,Accounting,8435,Diane Murphy
2,Gerard Bondur,Accounting,11472,Diane Murphy
3,Jeff Firrelli,Accounting,8992,Diane Murphy
4,Mary Patterson,Accounting,9998,Diane Murphy
5,William Patterson,Accounting,8870,Diane Murphy
6,Leslie Jennings,IT,8113,Leslie Thompson
7,Leslie Thompson,IT,5186,Leslie Thompson
8,Foon Yue Tseng,Sales,6660,George Vanauf
9,George Vanauf,Sales,10563,George Vanauf


In [44]:
# NTILE
pd.read_sql_query("""WITH productline_sales AS (
    SELECT productline,
           year(orderDate) order_year,
           ROUND(SUM(quantityOrdered * priceEach),0) order_value
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)
    GROUP BY productline, order_year
)
SELECT
    productline, 
    order_year, 
    order_value,
    NTILE(3) OVER (
        PARTITION BY order_year
        ORDER BY order_value DESC
    ) product_line_group
FROM 
    productline_sales""" , conn)

,productline,order_year,order_value,product_line_group
0,Classic Cars,2003,1374832.0,1
1,Vintage Cars,2003,619161.0,1
2,Trucks and Buses,2003,376657.0,1
3,Motorcycles,2003,348909.0,2
4,Planes,2003,309784.0,2
5,Ships,2003,222182.0,3
6,Trains,2003,65822.0,3
7,Classic Cars,2004,1763137.0,1
8,Vintage Cars,2004,854552.0,1
9,Motorcycles,2004,527244.0,1


In [45]:
# NTILE
pd.read_sql_query("""WITH productline_sales AS (
    SELECT productline,
           year(orderDate) order_year,
           ROUND(SUM(quantityOrdered * priceEach),0) order_value
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)
    GROUP BY productline, order_year
)
SELECT
    productline, 
    order_year, 
    order_value,
    NTILE(3) OVER (
        PARTITION BY order_year
    ) product_line_group
FROM 
    productline_sales""" , conn)

,productline,order_year,order_value,product_line_group
0,Vintage Cars,2003,619161.0,1
1,Classic Cars,2003,1374832.0,1
2,Trucks and Buses,2003,376657.0,1
3,Trains,2003,65822.0,2
4,Ships,2003,222182.0,2
5,Planes,2003,309784.0,3
6,Motorcycles,2003,348909.0,3
7,Classic Cars,2004,1763137.0,1
8,Vintage Cars,2004,854552.0,1
9,Trains,2004,96286.0,1


In [9]:
cursor.execute("""DROP TABLE if exists productLineSales""")
cursor.execute("""CREATE TABLE productLineSales
SELECT
    productLine,
    YEAR(orderDate) orderYear,
    quantityOrdered * priceEach orderValue
FROM
    orderDetails
        INNER JOIN
    orders USING (orderNumber)
        INNER JOIN
    products USING (productCode)
GROUP BY
    productLine ,
    YEAR(orderDate)""")


conn.commit()

In [10]:
# PERCENT_RANK
pd.read_sql_query("""SELECT
    productLine,
    orderYear,
    orderValue,
    ROUND(
    PERCENT_RANK()
    OVER (
        PARTITION BY orderYear
        ORDER BY orderValue
    ),2) percentile_rank
FROM
    productLineSales""" , conn)

,productLine,orderYear,orderValue,percentile_rank
0,Motorcycles,2003,2440.50,0.00
1,Trains,2003,2770.95,0.17
2,Trucks and Buses,2003,3284.28,0.33
3,Vintage Cars,2003,4080.00,0.50
4,Planes,2003,4825.44,0.67
5,Ships,2003,5072.71,0.83
6,Classic Cars,2003,5571.80,1.00
7,Motorcycles,2004,2598.77,0.00
8,Vintage Cars,2004,2819.28,0.17
9,Planes,2004,2857.35,0.33


In [12]:
# RANK
pd.read_sql_query("""SELECT
    sales_employee,
    fiscal_year,
    sale,
    RANK() OVER (PARTITION BY
                     fiscal_year
                 ORDER BY
                     sale DESC
                ) sales_rank
FROM
    sales""" , conn)

,sales_employee,fiscal_year,sale,sales_rank
0,John,2016,200.0,1
1,Alice,2016,150.0,2
2,Bob,2016,100.0,3
3,Bob,2017,150.0,1
4,John,2017,150.0,1
5,Alice,2017,100.0,3
6,John,2018,250.0,1
7,Alice,2018,200.0,2
8,Bob,2018,200.0,2


In [14]:
# RANK
pd.read_sql_query("""SELECT
    sales_employee,
    fiscal_year,
    sale,
    RANK() OVER (PARTITION BY
                     fiscal_year
                ) sales_rank
FROM
    sales""" , conn)

,sales_employee,fiscal_year,sale,sales_rank
0,Alice,2016,150.0,1
1,Bob,2016,100.0,1
2,John,2016,200.0,1
3,Alice,2017,100.0,1
4,Bob,2017,150.0,1
5,John,2017,150.0,1
6,Alice,2018,200.0,1
7,Bob,2018,200.0,1
8,John,2018,250.0,1


In [13]:
# RANK
pd.read_sql_query("""WITH order_values AS(
    SELECT 
        orderNumber, 
        YEAR(orderDate) order_year,
        quantityOrdered*priceEach AS order_value,
        RANK() OVER (
            PARTITION BY YEAR(orderDate)
            ORDER BY quantityOrdered*priceEach DESC
        ) order_value_rank
    FROM
        orders
    INNER JOIN orderDetails USING (orderNumber)
)
SELECT 
    * 
FROM 
    order_values
WHERE 
    order_value_rank <=3""" , conn)

,orderNumber,order_year,order_value,order_value_rank
0,10196,2003,9571.08,1
1,10206,2003,9568.73,2
2,10201,2003,9394.28,3
3,10312,2004,10286.40,1
4,10348,2004,9974.40,2
5,10304,2004,9467.68,3
6,10403,2005,11503.14,1
7,10405,2005,11170.52,2
8,10407,2005,10723.60,3


In [15]:
# ROW_NUMBER
pd.read_sql_query("""SELECT 
	ROW_NUMBER() OVER (
		ORDER BY productName
	) row_num,
    productName,
    msrp
FROM 
	products
ORDER BY 
	productName""" , conn)

,row_num,productName,msrp
0,1,18th century schooner,122.89
1,2,18th Century Vintage Horse Carriage,104.72
2,3,1900s Vintage Bi-Plane,68.51
3,4,1900s Vintage Tri-Plane,72.45
4,5,1903 Ford Model A,136.59
...,...,...,...
105,106,The Mayflower,86.61
106,107,The Queen Mary,99.31
107,108,The Schooner Bluenose,66.67
108,109,The Titanic,100.17


In [16]:
# ROW_NUMBER
pd.read_sql_query("""WITH inventory
AS (SELECT 
       productLine,
       productName,
       quantityInStock,
       ROW_NUMBER() OVER (
          PARTITION BY productLine 
          ORDER BY quantityInStock DESC) row_num
    FROM 
       products
   )
SELECT 
   productLine,
   productName,
   quantityInStock
FROM 
   inventory
WHERE 
   row_num <= 3""" , conn)



,productLine,productName,quantityInStock
0,Classic Cars,1995 Honda Civic,9772
1,Classic Cars,2002 Chevy Corvette,9446
2,Classic Cars,1976 Ford Gran Torino,9127
3,Motorcycles,2002 Suzuki XREO,9997
4,Motorcycles,1982 Ducati 996 R,9241
5,Motorcycles,1969 Harley Davidson Ultimate Chopper,7933
6,Planes,America West Airlines B757-200,9653
7,Planes,American Airlines: MD-11S,8820
8,Planes,ATA: B757-300,7106
9,Ships,The USS Constitution Ship,7083
